<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0110EN/notebook_images%20/cc-logo-square.png" width="200" alt="cognitiveclass.ai logo" />

<h1>Policy Iteration with Frozen Lake</h1> 

<h2>Table of Contents</h2>
<p>In this lab, you will use Perform Policy Iteration on the FrozenLake environment on open AI gym </p>

<ul>
    <li><a href="#PE">Policy Evaluation</a></li>
    <li><a href="#PI">Policy Improvement  </a></li>
    <li><a href="#Result">Policy Iteration</a></li>
</ul>
<p>Estimated Time Needed: <strong>25 min</strong></p>

<hr>

In [ ]:
#!conda install -c anaconda seaborn

#!pip install gym

In [ ]:

import gym
import matplotlib.pyplot as plt
import math 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
environment= gym.make('FrozenLake-v0')
environment.reset()
environment.env

In [ ]:
state_name=[['S','F','F','F'],
['F','H','F','H'],
['F','F','F','H'],
['H','F','F','G']]


<p>S : starting point, safe</p>
<p>F : frozen surface, safe</p>
<p>H : hole, fall to your doom</p>
<p>G : goal, where the frisbee is located</p>

<p>LEFT = 0</p>
<p>DOWN = 1</p>
<p>RIGHT = 2</p>
<p>UP = 3</p>

In [ ]:
for i in range(8):
    print("{},start{},stop{}".format(i,i%4,i%4+1) )

We create a frozen lake environment object, render it

In [ ]:
Environment=environment.env
Environment.render()

We obtain the number of states 

In [ ]:
Environment.nS

We also obtain the number of actions.

In [ ]:
Environment.nA

This function will plot out the states as a grid.

In [ ]:
class DrawStateGrid():
    def __init__(self,Environment,v=None):
        state_number=4
        self.state_name=[['S','F','F','F'],
        ['F','H','F','H'],
        ['F','F','F','H'],
        ['H','F','F','G']]
        self.numbber=[[0,1,2,3],
        [4,5,6,7],
        [8,9,10,11],
        [12,13,14,15]]

        self.state_action_state=np.zeros((Environment.nS,Environment.nA))
        
        
        for state in range(Environment.nS):
            for action in range(Environment.nA):
                if len(Environment.P[state][action])>1:
                    self.state_action_state[state,action]=Environment.P[state][action][1][1]
                else: 
                       self.state_action_state[state,action]=Environment.P[state][action][0][1]
        
        major_ticks=[i+1 for i in range(state_number)]
        minor_ticks=[i for i in range(state_number)]
        self.fig,self.ax = plt.subplots()


        self.ax.set_xlim([0,state_number])
        self.ax.set_ylim([0,state_number])
        self.ax.grid()
        self.ax.set_xticks(major_ticks)
        self.ax.set_xticks(minor_ticks, minor=True)
        self.ax.set_yticks(major_ticks)
        self.ax.set_yticks(minor_ticks, minor=True)
        n=0 
        for i,row in enumerate(self.state_name):
            for j,val in enumerate(row): 
                self.ax.text(j,4-i-1,"{}:{}".format(val,str(n)),size=25)
           
                n+=1
        
    
    def plot_state(self,start_state,end_state,action):

        #start_state=0
        #end_state=1
        #action=0
        #map states to position on table, top line on square subtract one and its the bottom line
        fig2 = plt.figure()
        y_top=[4,4,4,4,3,3,3,3,2,2,2,2,1,1,1,1]
        
        action_arrow=[(0.5,0),(0,-0.5),(-0.5,0),(0,0.5)]
        
        x_start=np.linspace(start_state%4,start_state%4+1,10)
        self.ax.fill_between(x_start,y_top[start_state]-1,y_top[start_state],color='b')
        
        x_stop=np.linspace(end_state%4,end_state%4+1,10)

        self.ax.fill_between( x_stop,y_top[end_state]-1,y_top[end_state],color='r')
        
     
        self.ax.arrow(x_start[5], y_top[start_state]-0.5, action_arrow[action][0], action_arrow[action][1],head_width=0.05, head_length=0.1, fc='k', ec='k') 
        
    def plot_policy(self,policy):
   
    
        action_arrow=[(-0.5,0),(0,-0.5),(0.5,0),(0,0.5)]
        y_top=[4,4,4,4,3,3,3,3,2,2,2,2,1,1,1,1]    
        for state,action_set in enumerate(policy):
            action=np.argmax(action_set)
            if max(action_set)==1:
                x_start=np.linspace(state%4,state%4+1,10)
                self.ax.arrow(x_start[5], y_top[state]-0.5, action_arrow[action][0], action_arrow[action][1],head_width=0.05, head_length=0.1, fc='k', ec='k')

we can draw the environment.

In [ ]:
DrawStateGrid(Environment)

<h2 id="PE">Policy Evaluation</h2> 

We will create an empty array for the 1-D value function and a  2D array for the policy. The policy will randomly select an action with equal Probability.

In [ ]:
V=np.zeros(Environment.nS)
policy=np.ones((Environment.nS,Environment.nA))/Environment.nA

This fucion will iteratively calculate the value function for the following policy 

$\mathcal{V^{k+1}}_{\pi}(s) = \sum_{a \in \mathcal{A}} \pi(a | s) \sum_{s' \in \mathcal{S}}  \mathcal{P}_{ss'}^a \big[\mathcal{R}_s^a + \gamma  {V^{k}}_{\pi}(s')\big]$

In [ ]:

#env, policy, gamma=1., theta=1e-8
def policy_evaluation(Environment,policy,max_iterations=100,discount_factor=1,theta=1e-9,record=False):
    V=np.zeros(Environment.nS)
    if record:
        delta_iteration=[]
        V_iterations=[[0 ] for i in range(Environment.nS) ]
        
    else:
        delta_iteration=None
        V_iterations=None
        

    discount_factor=1

    # for each iteration
    for iteration  in range(max_iterations):
        #stors the largest difference between states 
        delta = 0
        for state in range(Environment.nS):
            v=0
            for action,action_probability in enumerate(policy[state]):
                for state_probability, next_state, reward, terminated in Environment.P[state][action]:
                    v+=action_probability*state_probability*(reward+discount_factor*V[next_state])
                # find the largest difference between previous state estimate and present state estimate 
        
            delta=max(delta,v-V[state])
            if record:
                V_iterations[state].append(V[state])
                delta_iteration.append(delta)
            
            V[state]=v
    
    
        if delta<theta:
            break
            
    if record:
        return V, V_iterations, delta_iteration
    else:
        return V 
    



In [ ]:
max_iterations=100
discount_factor=1
theta=1e-9
V, V_iterations, delta_iteration=policy_evaluation(Environment,policy,max_iterations=100,discount_factor=1,theta=1e-9,record=True)

This cell will plot the value function for each iteration. It will also track the smallest change between each iteration and plot it.

In [ ]:
plt.subplot(211)
for s,v_iteration in enumerate(V_iterations):
    
    plt.plot(v_iteration,label="Return for state {}".format(s))

plt.ylabel('value for that state')
plt.xlabel('iterations k')
plt.subplot(212)
plt.plot(delta_iteration )
plt.plot(theta* np.ones(s))
plt.yscale('log')
plt.ylabel('delta log')   
plt.xlabel('iterations k')
plt.show()

We can plot the value function as a grid 

In [ ]:
Text = np.asarray([['V(0)=','V(1)=','V(2)=','V(3)='],['V(4)=','V(5)=','V(6)=','V(7)='],['V(8)=','V(9)=','V(10)=','V(11)='],['V(12)=','V(13)=','V(14)=','V(15)=']])
labels =(np.asarray(["{}{}".format(text,str(v)[0:4]) for v,text in zip(V,Text.flatten())]).reshape(4,4))

plt.figure(figsize=(8, 8))
sns.heatmap(V.reshape(4, 4),annot=labels,  cmap="YlGnBu", cbar=False,fmt='')
plt.show()

<h2 id="PI">Policy Improvement</h2> 

We create a function for the lookahead step.

$q_{\pi}(a,s)=\sum_{s' \in \mathcal{S}}  \mathcal{P}_{ss'}^a \big[\mathcal{R}_s^a + \gamma  {V^{k}}_{\pi}(s')\big]$

In [ ]:
def look_ahead(Environment, V,discount_factor=0.9):
    Q=np.zeros((Environment.nS,Environment.nA))
 
    for state in range(Environment.nS):

        for action in range(Environment.nA):
            q=0
            for state_probability, next_state, reward, terminated in Environment.P[state][action]: 
                q+=state_probability*(reward+discount_factor*V[next_state])
           
            Q[state,action]=q
    return Q

We can perform the lookahead step and plot the results.

In [ ]:
Q=look_ahead(Environment, V)   
plt.figure(figsize=(5, 16))
sns.heatmap(Q,  cmap="YlGnBu", annot=True, cbar=False);

We create a function the Performs Policy Improvement:

In [ ]:
def policy_impovment(Q,policy): 
    new_policy=policy
    N_actions=new_policy[0,:].shape[0]

    not_terminal_states=np.sum(Q,axis=1)!=0
    best_actions=np.argmax(Q[:,:],axis=1)

    for state,(not_terminal,best_action) in enumerate(zip(not_terminal_states,best_actions)):
            if not_terminal==True:
                new_policy[state] =np.eye(N_actions)[best_action]
            else:
                 new_policy[state]=policy[state]
    return new_policy

In [ ]:
new_policy=policy_impovment(Q,policy)

In [ ]:
plt.figure(figsize=(5, 16))
sns.heatmap(new_policy,  cmap="YlGnBu", annot=True, cbar=False);

The following functions perform the same task but work with deterministic froze lake [2].

In [ ]:
def q_value(env, V, s, gamma=0.9):
    r"""Q-value (action-value) function from state-value function

    Returns Q values, values of actions.

    Args:
        env (gym.env): OpenAI environment class instantiated and assigned to an object.
        V (np.array): array of state-values obtained from policy evaluation function.
        s (integer): integer representing current state in the gridworld
        gamma (float): discount rate for rewards.
    """
    # 1. Create q-value array for one state
    # We have 4 actions, so let's create an array with the size of 4
    A_n=env.nA
    q = np.zeros(A_n)

    # 2. Loop through each action
    for a in range(A_n):
        # 2.1 For each action, we've our transition probabilities, next state, rewards and whether the game ended
        for prob, next_state, reward, done in env.P[s][a]:
            # 2.1.1 Get our action-values from state-values
            q[a] += prob * (reward + gamma * V[next_state])

    # Return action values
    return q



def policy_improvement(env, V, gamma=1):

    # 1. Blank policy
    policy = np.zeros([env.nS, env.nA]) / env.nA

    # 2. For each state in 16 states
    for s in range(env.nS):

        # 2.1 Get q values: q.shape returns (4,)
        q = q_value(env, V, s, gamma=0.9)


        best_a = np.argwhere(q==np.max(q)).flatten()

        # 2.3 One-hot encode be
        
        
        policy[s] = np.sum([np.eye(env.nA)[i] for i in best_a], axis=0)/len(best_a)

    return policy 

<h2 id="PI">Policy Iteration</h2> 

Policy Iteration we can now 

In [ ]:




def policy_iteration(env, discount_factor=0.9, theta=1e-9,gamma=1.):
    # 1. Create equiprobable policy where every state has 4 actions with equal probabilities as a starting policy
    policy = np.ones([env.nS, env.nA]) / env.nA

    # 2. Loop through policy_evaluation and policy_improvement functions
    while True:
        # 2.1 Get state-values
        V=policy_evaluation(Environment,policy,max_iterations=100,discount_factor=gamma,theta=1e-9)
      
            
 
        # 2.2 Get new policy by getting q-values and maximizing q-values per state to get best action per state
        new_policy= policy_improvement(env, V)
        #new_policy=policy_impovment(Q)
        # 2.3 Stop if the value function estimates for successive policies has converged
        if np.max(abs(policy_evaluation(env, policy) - policy_evaluation(env, new_policy))) < theta * 1e2:
            break;

        policy = new_policy
    return policy, V ,Q


We perform Policy Iteration  and plot out the policy.

In [ ]:
policy_pi, V_pi ,q_pi= policy_iteration(Environment)
test=DrawStateGrid(Environment,V_pi)
test.plot_policy(policy_pi)

This result seems strange;let's look at the value function and action function. 

In [ ]:
sns.heatmap(V_pi.reshape(4,4),annot=True, cbar=False)
plt.title("Value function ")
plt.show()
sns.heatmap(q_pi,  cmap="YlGnBu", annot=True, cbar=False)
plt.title("Action function")
plt.show()

To understand, let's look at the portability of each action.

Right

In [ ]:
Environment.P[14][2]

Down

In [ ]:
Environment.P[14][1]

We can also determine the best policy for the deterministic case.

In [ ]:
from deterministicfrozenlake import FrozenLakeEnv

In [ ]:
env = FrozenLakeEnv(is_slippery=False)

In [ ]:
policy_pi, V_pi ,q_pi= policy_iteration(env)

In [ ]:
test=DrawStateGrid(Environment,V_pi)
test.plot_policy(policy_pi)

<b>About the Authors:</b> 

<a href="https://www.linkedin.com/in/joseph-s-50398b136/">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD.

<b>References</b>

<ol type="1">
    <li>Sutton, Richard S., and Andrew G. Barto. "Reinforcement learning: An introduction." 2011></li>
    <li><a href="https://www.deeplearningwizard.com/deep_learning/deep_reinforcement_learning_pytorch/dynamic_programming_frozenlake/">Dynamic Programming-Deep Learning Wizard</a></li>
    <li><a href="https://gym.openai.com/envs/FrozenLake-v0/">FrozenLake-v0</a></li>
</ol>

Copyright &copy; 2020 <a href="cognitiveclass.ai?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu">cognitiveclass.ai</a>. This notebook and its source code are released under the terms of the <a href="https://bigdatauniversity.com/mit-license/">MIT License</a>.